# Pinecone DB (프리티어 사용)

In [1]:
!pip install pinecone

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_6yQ1PT_6emC5ry7ZCvtLfgFGpvMJsUAsR3B3cTdquQYZkemddrQBGrERJsXFhuKbG43UNM")
index_name = "reviewtest"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-ynau2d6.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
reviews = [
    {"id": "1", "text": "성능이 빠르고 부팅 속도도 빨라서 만족스러워요!", "sentiment": "positive"},
    {"id": "2", "text": "배터리가 너무 빨리 닳아서 외출할 땐 불편해요.", "sentiment": "negative"},
    {"id": "3", "text": "화면이 선명하고 색감도 좋아서 디자인 작업에 딱이에요.", "sentiment": "positive"},
    {"id": "4", "text": "팬 소음이 생각보다 심해서 집중이 잘 안 돼요.", "sentiment": "negative"},
    {"id": "5", "text": "가볍고 휴대성이 좋아서 어디서든 쓰기 좋아요.", "sentiment": "positive"},
    {"id": "6", "text": "처음엔 괜찮았는데 며칠 쓰고 나서 버벅거려요.", "sentiment": "negative"},
    {"id": "7", "text": "터치패드 반응도 좋고 키보드 타건감도 만족스럽네요.", "sentiment": "positive"},
    {"id": "8", "text": "인터넷 연결이 자주 끊겨서 스트레스 받아요.", "sentiment": "negative"},
    {"id": "9", "text": "영상 시청할 때 화질과 사운드가 모두 뛰어나요!", "sentiment": "positive"},
    {"id": "10", "text": "발열이 심해서 오래 쓰기가 힘들어요.", "sentiment": "negative"},
    {"id": "11", "text": "가성비 최고예요. 이 가격에 이런 성능이라니!", "sentiment": "positive"},
    {"id": "12", "text": "충전기가 너무 크고 무거워서 들고 다니기 불편해요.", "sentiment": "negative"},
    {"id": "13", "text": "부팅도 빠르고 여러 프로그램 동시에 켜도 무리 없어요.", "sentiment": "positive"},
    {"id": "14", "text": "배송 상태가 안 좋아서 기스가 있었어요.", "sentiment": "negative"},
    {"id": "15", "text": "디자인이 세련되고 얇아서 들고 다니기 좋아요.", "sentiment": "positive"},
    {"id": "16", "text": "OS 업데이트 이후에 자꾸 오류가 생기네요.", "sentiment": "negative"},
    {"id": "17", "text": "화면 크기가 커서 작업할 때 정말 편리해요.", "sentiment": "positive"},
    {"id": "18", "text": "소프트웨어 호환 문제가 많아서 자주 멈춰요.", "sentiment": "negative"},
    {"id": "19", "text": "프리미엄 모델답게 전반적으로 퀄리티가 높아요.", "sentiment": "positive"},
    {"id": "20", "text": "AS 서비스가 별로라서 실망했어요.", "sentiment": "negative"}
]

In [4]:
# Pinecone index 및 임베딩 모델 로드드
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key="pcsk_6yQ1PT_6emC5ry7ZCvtLfgFGpvMJsUAsR3B3cTdquQYZkemddrQBGrERJsXFhuKbG43UNM")
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# index에 리뷰 데이터 추가
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (
            review['id'],
            review_embed,
            {
                'text': review['text'],
                'sentiment': review['sentiment']
            }
        )
    ])

### 리뷰 데이터 조회

In [6]:
query_text = "이 노트북의 성능은 괜찮은가요?"
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

results

{'matches': [{'id': '14',
              'metadata': {'sentiment': 'negative',
                           'text': '배송 상태가 안 좋아서 기스가 있었어요.'},
              'score': 0.764045238,
              'values': []},
             {'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '성능이 빠르고 부팅 속도도 빨라서 만족스러워요!'},
              'score': 0.728726566,
              'values': []},
             {'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '배터리가 너무 빨리 닳아서 외출할 땐 불편해요.'},
              'score': 0.711406112,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [9]:
for review in results['matches']:
    metadata = review['metadata']
    print(metadata['text'], f'(감성: {metadata["sentiment"]})')

배송 상태가 안 좋아서 기스가 있었어요. (감성: negative)
성능이 빠르고 부팅 속도도 빨라서 만족스러워요! (감성: positive)
배터리가 너무 빨리 닳아서 외출할 땐 불편해요. (감성: negative)


### 리뷰 감성 분석석

In [10]:
query_text = "배터리도 빨리 닳고 벽돌보다 무거워요. 비추천"
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True
)

results

{'matches': [{'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '성능이 빠르고 부팅 속도도 빨라서 만족스러워요!'},
              'score': 0.909920573,
              'values': []},
             {'id': '6',
              'metadata': {'sentiment': 'negative',
                           'text': '처음엔 괜찮았는데 며칠 쓰고 나서 버벅거려요.'},
              'score': 0.846480906,
              'values': []},
             {'id': '7',
              'metadata': {'sentiment': 'positive',
                           'text': '터치패드 반응도 좋고 키보드 타건감도 만족스럽네요.'},
              'score': 0.816115379,
              'values': []},
             {'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '배터리가 너무 빨리 닳아서 외출할 땐 불편해요.'},
              'score': 0.808998048,
              'values': []},
             {'id': '12',
              'metadata': {'sentiment': 'negative',
                           'text': '충전기가 너무 크고 무거워서 들고 다니기 불편해요.'},
              'score'

In [11]:
sentiment_counts = {
    "positive": 0,
    "negative": 0
}

for review in results['matches']:
    sentiment = review['metadata']["sentiment"]
    sentiment_counts[sentiment] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: negative
